In [2]:
import sys
sys.path.append('..')
import pysimARG.test

In [4]:
pysimARG.test.array1, pysimARG.test.array2, pysimARG.test.matrix1

(array([1, 2, 3]),
 array([4, 5, 6]),
 array([[1, 2],
        [3, 4]]))